## 1. Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from obspy import read, UTCDateTime
from obspy.clients.fdsn import Client

# Import local modules
import sys
sys.path.insert(0, '..')

from src.models import SeismicCNN, save_model, load_model
from src.data import SeismicDataProcessor
from src.utils import plot_seismogram, get_device, set_random_seed
from src.detect import multi_class_detection

# Set random seed for reproducibility
set_random_seed(42)

## 2. Initialize Device and Model

In [ ]:
# Get appropriate device (GPU if available, otherwise CPU)
device = get_device()
print(f"Using device: {device}")

# Create model
model = SeismicCNN(input_channels=3, num_classes=2, dropout=0.5)
model = model.to(device)
print(f"\nModel architecture:\n{model}")

## 3. Create Synthetic Seismogram

For demonstration purposes, we'll create a synthetic seismogram.

In [ ]:
from obspy import Trace, Stream
from obspy.core import UTCDateTime

# Create synthetic data
sampling_rate = 100.0  # Hz
duration = 60.0  # seconds
npts = int(sampling_rate * duration)

# Generate synthetic seismogram with noise and a signal
t = np.linspace(0, duration, npts)

# Add noise
np.random.seed(42)
data_z = np.random.randn(npts) * 0.1
data_n = np.random.randn(npts) * 0.1
data_e = np.random.randn(npts) * 0.1

# Add a synthetic event at t=30s
event_time = 30.0
event_start = int(event_time * sampling_rate)
event_duration = int(5.0 * sampling_rate)
event_signal = np.exp(-np.linspace(0, 5, event_duration)) * np.sin(2 * np.pi * 5 * np.linspace(0, 5, event_duration))

data_z[event_start:event_start+event_duration] += event_signal * 2
data_n[event_start:event_start+event_duration] += event_signal * 1.5
data_e[event_start:event_start+event_duration] += event_signal * 1.0

# Create ObsPy Stream
starttime = UTCDateTime("2025-01-01T00:00:00")

tr_z = Trace(data=data_z)
tr_z.stats.sampling_rate = sampling_rate
tr_z.stats.starttime = starttime
tr_z.stats.network = "UW"
tr_z.stats.station = "TEST"
tr_z.stats.channel = "HHZ"

tr_n = Trace(data=data_n)
tr_n.stats.sampling_rate = sampling_rate
tr_n.stats.starttime = starttime
tr_n.stats.network = "UW"
tr_n.stats.station = "TEST"
tr_n.stats.channel = "HHN"

tr_e = Trace(data=data_e)
tr_e.stats.sampling_rate = sampling_rate
tr_e.stats.starttime = starttime
tr_e.stats.network = "UW"
tr_e.stats.station = "TEST"
tr_e.stats.channel = "HHE"

stream = Stream([tr_z, tr_n, tr_e])

print(f"Created synthetic stream with {len(stream)} traces")
print(stream)

## 4. Plot the Seismogram

In [ ]:
fig, axes = plot_seismogram(stream, figsize=(14, 6))
plt.show()

## 5. Process Data

In [ ]:
# Initialize data processor
processor = SeismicDataProcessor(
    sampling_rate=100.0,
    window_length=30.0,
    normalize=True
)

# Preprocess the stream
stream_processed = processor.preprocess_stream(
    stream.copy(),
    freqmin=1.0,
    freqmax=20.0
)

# Convert to array
data_array = processor.stream_to_array(stream_processed)
print(f"Processed data shape: {data_array.shape}")

## 6. Make Predictions with Model

Note: This is using an untrained model for demonstration.

In [ ]:
# Prepare data for model
data_tensor = processor.to_torch(data_array).unsqueeze(0).to(device)
print(f"Input tensor shape: {data_tensor.shape}")

# Make prediction
model.eval()
with torch.no_grad():
    output = model(data_tensor)
    probabilities = torch.softmax(output, dim=1)

print(f"\nModel output shape: {output.shape}")
print(f"Predictions (probabilities): {probabilities.cpu().numpy()}")
print(f"Predicted class: {torch.argmax(probabilities, dim=1).item()}")

## 7. Summary

This notebook demonstrated:
1. Setting up the environment and loading modules
2. Creating and configuring a model
3. Creating synthetic seismic data
4. Visualizing seismograms
5. Processing seismic data
6. Making predictions with a neural network

Next steps:
- Train the model on real labeled data
- Use SeisBench models for event detection
- Analyze real debris flow events